# Collect Census Data

Socioeconomic data is gathered from the Census API using the CensusData package ([documentation here](https://jtleider.github.io/censusdata/index.html)). I will be using the 2019 Amerian Community Survey 5-year estimates.

I reference Stanford's Deep Solar analysis for which variables should be collected. The list of variables can be found in `supplementary_info.pdf` in references/. 

I collect data for all ZCTAs in each state covered by Tracking the Sun report. 

In [1]:
import os
import json
import censusdata
import pandas as pd
import numpy as np

The Census uses FIPS Codes to identify states, so I won't be able to use their names. [This](https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html) page was used to determine the FIPS code associated with each state I am using. Below I use pandas to create a dataframe of the table.

In [2]:
tables = pd.read_html('https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html')

In [3]:
STATE_CODES = tables[0]
STATE_CODES.head()

,Name,FIPS State Numeric Code,Official USPS Code
0,Alabama,1,AL
1,Alaska,2,AK
2,Arizona,4,AZ
3,Arkansas,5,AR
4,California,6,CA


I only need data for 20 states (19 and DC), below is a list of those states.

In [4]:
STATES = ['arizona', 'new_jersey', 'connecticut', 'texas', 'washington', 'ohio', 'florida', 'new_york',
            'oregon', 'illinois', 'district_of_columbia', 'massachusetts', 'new_hampshire', 'california',
            'new_mexico', 'pennsylvania', 'vermont', 'delaware', 'utah', 'wisconsin']

In order to filter `STATE_CODES` for these states I need to reformat the `Name` column to be lowercase and replace any spaces with an underscore.

In [5]:
STATE_CODES['name'] = STATE_CODES['Name'].apply(lambda n: n.lower().replace(' ', '_'))
STATE_CODES.tail()

,Name,FIPS State Numeric Code,Official USPS Code,name
46,Virginia,51,VA,virginia
47,Washington,53,WA,washington
48,West Virginia,54,WV,west_virginia
49,Wisconsin,55,WI,wisconsin
50,Wyoming,56,WY,wyoming


In [6]:
# Filter for state names in STATES list
valid_states = STATE_CODES[STATE_CODES['name'].isin(STATES)]
valid_states.shape

(20, 4)

In [7]:
valid_codes = list(valid_states['FIPS State Numeric Code'])
valid_codes

[4, 6, 9, 10, 11, 12, 17, 25, 33, 34, 35, 36, 39, 41, 42, 48, 49, 50, 53, 55]

Now that I have all of the codes, I need to determine the correct variable names to call from the API. All of the census variables used by Deep Solar can be found in `deepsolar_meta.csv` in references/. `ACS2019_Table_Shells.xlsx` in data/ is essentially the data dictionary for the 2019 5 year ACS estimates. These two will be used to determine the correct variable names which will be listed below in a dictionary. `censusdata.censusvar` can be used to search variables for different Census Surveys and I am using it here to ensure all of the variables are correct. [This](https://data.census.gov/cedsci/all?d=ACS%205-Year%20Estimates%20Detailed%20Tables) page was also used to search for variable names

In [8]:
censusdata.censusvar('acs5', 2019, ['B19081_002E'])

{'B19081_002E': ['MEAN HOUSEHOLD INCOME OF QUINTILES',
  'Estimate!!Quintile Means:!!Second Quintile',
  'int']}

In [9]:
CENSUS_VARIABLES = {'B19083_001E': 'gini_index', 'B01003_001E': 'total_population', 'B15003_022E': 'education_bachelor', 'B15003_023E': 'education_master', 'B15003_024E': 'education_professional_school', 'B15003_025E': 'education_doctoral',
                    'B15003_001E': 'education_population', 'B15003_017E': 'education_high_school_diploma', 'B15003_018E': 'education_GED', 'B15003_019E': 'education_some_college_less_than_year', 'B15003_020E': 'education_some_college_no_degree',
                    'B15003_021E': 'education_associates', 'B24124_001E': 'employed', 'B19013_001E': 'median_household_income', 'B02001_002E': 'race_white', 'B02001_003E': 'race_black_africa', 'B02001_004E': 'race_india_alaska',
                    'B02001_005E': 'race_asian', 'B02001_006E': 'race_islander', 'B02001_007E': 'race_other', 'B02001_008E': 'race_two_or_more', 'B09019_001E': 'household_count', 'B01001_003E': 'male_under_5', 'B01001_004E': 'male_5_to_9',
                    'B01001_005E': 'male_10_to_14', 'B01001_006E': 'male_15_to_17', 'B01001_007E': 'male_18_to_19', 'B01001_008E': 'male_20', 'B01001_009E': 'male_21', 'B01001_010E': 'male_22_to_24', 'B01001_011E': 'male_25_to_29', 
                    'B01001_012E': 'male_30_to_34', 'B01001_013E': 'male_35_to_39', 'B01001_014E': 'male_40_to_44', 'B01001_015E': 'male_45_to_49', 'B01001_016E': 'male_50_to_54', 'B01001_017E': 'male_55_to_59', 'B01001_018E': 'male_60_to_61',
                    'B01001_019E': 'male_62_to_64', 'B01001_020E': 'male_65_to_66', 'B01001_021E': 'male_67_to_69', 'B01001_022E': 'male_70_to_74', 'B01001_023E': 'male_75_to_79', 'B01001_024E': 'male_80_to_84', 'B01001_025E': 'male_85_over',
                    'B01001_027E': 'female_under_5', 'B01001_028E': 'female_5_to_9', 'B01001_029E': 'female_10_to_14', 'B01001_030E': 'female_15_to_17', 'B01001_031E': 'female_18_to_19', 'B01001_032E': 'female_20', 
                    'B01001_033E': 'female_21', 'B01001_034E': 'female_22_to_24', 'B01001_035E': 'female_25_to_29', 'B01001_036E': 'female_30_to_34', 'B01001_037E': 'female_35_to_39', 'B01001_038E': 'female_40_to_44',
                    'B01001_039E': 'female_45_to_49', 'B01001_040E': 'female_50_to_54', 'B01001_041E': 'female_55_to_59', 'B01001_042E': 'female_60_to_61', 'B01001_043E': 'female_62_to_64', 'B01001_044E': 'female_65_to_66',
                    'B01001_045E': 'female_67_to_69', 'B01001_046E': 'female_70_to_74', 'B01001_047E': 'female_75_to_79', 'B01001_048E': 'female_80_to_84', 'B01001_049E': 'female_85_over', 'B08101_009E': 'transportation_drove_alone',
                    'B08101_017E': 'transportation_carpool', 'B08101_025E': 'transportation_public', 'B08101_033E': 'transportation_walked', 'B08101_041E': 'transportation_other', 'B08101_049E': 'transportation_worked_from_home',
                    'B08101_001E': 'transportation_total', 'B08303_001E': 'travel_time_total', 'B08303_002E': 'travel_time_under_5', 'B08303_003E': 'travel_time_5_to_9', 'B08303_004E': 'travel_time_10_to_14', 'B08303_005E': 'travel_time_15_to_19',
                    'B08303_006E': 'travel_time_20_to_24', 'B08303_007E': 'travel_time_25_to_29', 'B08303_008E': 'travel_time_30_to_34', 'B08303_009E': 'travel_time_35_to_39', 'B08303_010E': 'travel_time_40_to_44',
                    'B08303_011E': 'travel_time_45_to_59', 'B08303_012E': 'travel_time_60_to_89', 'B08303_013E': 'travel_time_90_or_more', 'B19301_001E': 'per_capita_income', 'B25040_001E': 'heating_fuel_housing_unit_count', 'B25040_002E': 'heating_fuel_gas',
                    'B25040_003E': 'heating_fuel_bottled_tank_lp_gas', 'B25040_004E': 'heating_fuel_electricity', 'B25040_005E': 'heating_fuel_fuel_oil_kerosene', 'B25040_006E': 'heating_fuel_coal_coke', 'B25040_007E': 'heating_fuel_wood',
                    'B25040_008E': 'heating_fuel_solar', 'B25040_009E': 'heating_fuel_other', 'B25040_010E': 'heating_fuel_none', 'B25002_001E': 'housing_unit_count', 'B25002_002E': 'housing_unit_occupied_count', 'B25064_001E': 'housing_unit_median_gross_rent',
                    'B25077_001E': 'housing_unit_median_value', 'B01002_001E': 'age_median', 'B19025_001E': 'aggregate_household_income', 'B19081_001E': 'mean_household_income_lowest_quintile', 'B19081_002E': 'mean_household_income_second_quintile',
                    'B19081_003E': 'mean_household_income_third_quintile', 'B19081_004E': 'mean_household_income_fourth_quintile', 'B19081_005E': 'mean_household_income_highest_quintile', 'B19081_006E': 'mean_household_income_top_5_percent'} 

This covers the vast majority of variables used for Deep Solar. I excluded the specific occupation rates for now. Land area and water area are actually in the geographic json data. There are a few variables I will need to calculate for the data collected above. Now I need to collect this data for each ZCTA in the states covered by the PV systems data. I am going to start with a single state before iterating over all state codes in `valid_codes`.

In [10]:
# Keys of CENSUS_VARIABLES (actual variable names from census API)
vars_to_collect = list(CENSUS_VARIABLES.keys())

In [11]:
# Collect data for california
censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '06'), ('zip code tabulation area', '*')]), vars_to_collect).rename(columns=CENSUS_VARIABLES)

,gini_index,total_population,education_bachelor,education_master,education_professional_school,education_doctoral,education_population,education_high_school_diploma,education_GED,education_some_college_less_than_year,...,housing_unit_median_gross_rent,housing_unit_median_value,age_median,aggregate_household_income,mean_household_income_lowest_quintile,mean_household_income_second_quintile,mean_household_income_third_quintile,mean_household_income_fourth_quintile,mean_household_income_highest_quintile,mean_household_income_top_5_percent
"ZCTA5 93252: Summary level: unknown, state:06> zip code tabulation area:93252",0.4468,2488,104,12,23,0,1851,438,202,216,...,795,103800,38.9,2.722380e+07,9456.0,22143.0,36980.0,52597.0,118049.0,202422.0
"ZCTA5 93270: Summary level: unknown, state:06> zip code tabulation area:93270",0.5705,5367,238,55,0,0,3266,544,120,217,...,808,194900,35.3,1.064625e+08,8105.0,18628.0,34856.0,67221.0,198364.0,378263.0
"ZCTA5 93291: Summary level: unknown, state:06> zip code tabulation area:93291",0.4724,59270,4966,1922,665,392,34651,5838,783,2386,...,994,267200,30.4,1.484341e+09,12934.0,35726.0,62902.0,99091.0,215173.0,349582.0
"ZCTA5 93304: Summary level: unknown, state:06> zip code tabulation area:93304",0.4389,49115,1470,567,207,158,28261,7976,1130,1969,...,952,158600,29.2,7.982409e+08,10715.0,26416.0,41163.0,59182.0,128941.0,230632.0
"ZCTA5 93306: Summary level: unknown, state:06> zip code tabulation area:93306",0.4577,72280,4509,2162,459,186,42605,9468,1557,3305,...,1032,196800,31.3,1.651396e+09,13575.0,33879.0,55820.0,87598.0,186763.0,320931.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"ZCTA5 96108: Summary level: unknown, state:06> zip code tabulation area:96108",0.2777,80,0,0,0,0,80,38,0,22,...,-666666666,44600,74.5,3.080300e+06,NaN,NaN,NaN,NaN,NaN,NaN
"ZCTA5 96110: Summary level: unknown, state:06> zip code tabulation area:96110",0.3542,126,13,3,0,0,108,27,0,15,...,-666666666,-666666666,55.8,4.010800e+06,NaN,NaN,NaN,NaN,NaN,NaN
"ZCTA5 96115: Summary level: unknown, state:06> zip code tabulation area:96115",0.3565,115,31,0,0,0,112,15,0,22,...,-666666666,171600,68.2,2.885200e+06,NaN,NaN,NaN,NaN,NaN,NaN
"ZCTA5 96116: Summary level: unknown, state:06> zip code tabulation area:96116",0.1553,172,0,0,0,0,172,90,0,27,...,-666666666,2000001,65.1,7.419200e+06,NaN,NaN,NaN,NaN,NaN,NaN


That worked. Now I can repeat this process for the rest of the states. First I need to adjust `valid_codes`. The values should be strings, and any single digit code needs a leading zero.

In [12]:
valid_codes_str = [str(c) for c in valid_codes]

In [14]:
valid_codes_str[:3] = ['04', '06', '09']
valid_codes_str

['04',
 '06',
 '09',
 '10',
 '11',
 '12',
 '17',
 '25',
 '33',
 '34',
 '35',
 '36',
 '39',
 '41',
 '42',
 '48',
 '49',
 '50',
 '53',
 '55']

For each state I will append the resulting dataframe to a list and merge them all at the end.

In [15]:
DF_LIST = []
for state_code in valid_codes_str:
    api_resp = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', state_code), ('zip code tabulation area', '*')]), vars_to_collect)
    DF_LIST.append(api_resp)

In [16]:
len(DF_LIST)

20

In [17]:
# Concat into one main dataframe
main_df = pd.concat(DF_LIST)

In [18]:
main_df.head()

,B19083_001E,B01003_001E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B15003_001E,B15003_017E,B15003_018E,B15003_019E,...,B25064_001E,B25077_001E,B01002_001E,B19025_001E,B19081_001E,B19081_002E,B19081_003E,B19081_004E,B19081_005E,B19081_006E
"ZCTA5 85610: Summary level: unknown, state:04> zip code tabulation area:85610",0.3893,1071,56,20,6,2,721,217,18,50,...,565,118200,52.5,24224900.0,15735.0,28976.0,41584.0,60403.0,121871.0,186273.0
"ZCTA5 85614: Summary level: unknown, state:04> zip code tabulation area:85614",0.4224,23777,5034,2577,645,498,21588,3984,416,1997,...,1011,174800,70.7,882515500.0,15092.0,33942.0,52059.0,78902.0,156740.0,271955.0
"ZCTA5 85624: Summary level: unknown, state:04> zip code tabulation area:85624",0.4391,1289,226,164,46,0,897,222,16,10,...,719,240000,51.1,30288800.0,12085.0,26596.0,40793.0,63481.0,139590.0,195939.0
"ZCTA5 85629: Summary level: unknown, state:04> zip code tabulation area:85629",0.3565,25770,3482,1696,302,140,16121,3035,526,1320,...,1410,194200,35.2,784399700.0,24218.0,55100.0,82356.0,109225.0,187332.0,290584.0
"ZCTA5 85630: Summary level: unknown, state:04> zip code tabulation area:85630",0.5500,1757,189,74,55,0,1395,349,39,116,...,493,163600,58.3,53583600.0,6123.0,16639.0,37332.0,58660.0,167178.0,298053.0


In [19]:
main_df.shape

(15740, 111)

Replace column names with more specific variable names

In [20]:
main_df = main_df.rename(columns=CENSUS_VARIABLES)
main_df.head()

,gini_index,total_population,education_bachelor,education_master,education_professional_school,education_doctoral,education_population,education_high_school_diploma,education_GED,education_some_college_less_than_year,...,housing_unit_median_gross_rent,housing_unit_median_value,age_median,aggregate_household_income,mean_household_income_lowest_quintile,mean_household_income_second_quintile,mean_household_income_third_quintile,mean_household_income_fourth_quintile,mean_household_income_highest_quintile,mean_household_income_top_5_percent
"ZCTA5 85610: Summary level: unknown, state:04> zip code tabulation area:85610",0.3893,1071,56,20,6,2,721,217,18,50,...,565,118200,52.5,24224900.0,15735.0,28976.0,41584.0,60403.0,121871.0,186273.0
"ZCTA5 85614: Summary level: unknown, state:04> zip code tabulation area:85614",0.4224,23777,5034,2577,645,498,21588,3984,416,1997,...,1011,174800,70.7,882515500.0,15092.0,33942.0,52059.0,78902.0,156740.0,271955.0
"ZCTA5 85624: Summary level: unknown, state:04> zip code tabulation area:85624",0.4391,1289,226,164,46,0,897,222,16,10,...,719,240000,51.1,30288800.0,12085.0,26596.0,40793.0,63481.0,139590.0,195939.0
"ZCTA5 85629: Summary level: unknown, state:04> zip code tabulation area:85629",0.3565,25770,3482,1696,302,140,16121,3035,526,1320,...,1410,194200,35.2,784399700.0,24218.0,55100.0,82356.0,109225.0,187332.0,290584.0
"ZCTA5 85630: Summary level: unknown, state:04> zip code tabulation area:85630",0.5500,1757,189,74,55,0,1395,349,39,116,...,493,163600,58.3,53583600.0,6123.0,16639.0,37332.0,58660.0,167178.0,298053.0


Reset index to extract ZCTA and state code.

In [21]:
main_df = main_df.reset_index()
main_df.head()

,index,gini_index,total_population,education_bachelor,education_master,education_professional_school,education_doctoral,education_population,education_high_school_diploma,education_GED,...,housing_unit_median_gross_rent,housing_unit_median_value,age_median,aggregate_household_income,mean_household_income_lowest_quintile,mean_household_income_second_quintile,mean_household_income_third_quintile,mean_household_income_fourth_quintile,mean_household_income_highest_quintile,mean_household_income_top_5_percent
0,"ZCTA5 85610: Summary level: unknown, state:04>...",0.3893,1071,56,20,6,2,721,217,18,...,565,118200,52.5,24224900.0,15735.0,28976.0,41584.0,60403.0,121871.0,186273.0
1,"ZCTA5 85614: Summary level: unknown, state:04>...",0.4224,23777,5034,2577,645,498,21588,3984,416,...,1011,174800,70.7,882515500.0,15092.0,33942.0,52059.0,78902.0,156740.0,271955.0
2,"ZCTA5 85624: Summary level: unknown, state:04>...",0.4391,1289,226,164,46,0,897,222,16,...,719,240000,51.1,30288800.0,12085.0,26596.0,40793.0,63481.0,139590.0,195939.0
3,"ZCTA5 85629: Summary level: unknown, state:04>...",0.3565,25770,3482,1696,302,140,16121,3035,526,...,1410,194200,35.2,784399700.0,24218.0,55100.0,82356.0,109225.0,187332.0,290584.0
4,"ZCTA5 85630: Summary level: unknown, state:04>...",0.5500,1757,189,74,55,0,1395,349,39,...,493,163600,58.3,53583600.0,6123.0,16639.0,37332.0,58660.0,167178.0,298053.0


In [22]:
main_df['index'][0].params()

(('state', '04'), ('zip code tabulation area', '85610'))

The original index is actually a `censusgeo` object, so I need to call the `params()` method to return the geographic values as a tuple, from which I can index and get the data I need.

In [23]:
main_df['state_code'] = main_df['index'].apply(lambda c: c.params()[0][1])
main_df['zcta'] = main_df['index'].apply(lambda c: c.params()[1][1])
main_df.head()

,index,gini_index,total_population,education_bachelor,education_master,education_professional_school,education_doctoral,education_population,education_high_school_diploma,education_GED,...,age_median,aggregate_household_income,mean_household_income_lowest_quintile,mean_household_income_second_quintile,mean_household_income_third_quintile,mean_household_income_fourth_quintile,mean_household_income_highest_quintile,mean_household_income_top_5_percent,state_code,zcta
0,"ZCTA5 85610: Summary level: unknown, state:04>...",0.3893,1071,56,20,6,2,721,217,18,...,52.5,24224900.0,15735.0,28976.0,41584.0,60403.0,121871.0,186273.0,04,85610
1,"ZCTA5 85614: Summary level: unknown, state:04>...",0.4224,23777,5034,2577,645,498,21588,3984,416,...,70.7,882515500.0,15092.0,33942.0,52059.0,78902.0,156740.0,271955.0,04,85614
2,"ZCTA5 85624: Summary level: unknown, state:04>...",0.4391,1289,226,164,46,0,897,222,16,...,51.1,30288800.0,12085.0,26596.0,40793.0,63481.0,139590.0,195939.0,04,85624
3,"ZCTA5 85629: Summary level: unknown, state:04>...",0.3565,25770,3482,1696,302,140,16121,3035,526,...,35.2,784399700.0,24218.0,55100.0,82356.0,109225.0,187332.0,290584.0,04,85629
4,"ZCTA5 85630: Summary level: unknown, state:04>...",0.5500,1757,189,74,55,0,1395,349,39,...,58.3,53583600.0,6123.0,16639.0,37332.0,58660.0,167178.0,298053.0,04,85630


Now I can drop the `index` column and save this dataframe as a csv for safe keeping. `data/base_census.csv`

In [24]:
main_df = main_df.drop('index', axis=1)
main_df.head()

,gini_index,total_population,education_bachelor,education_master,education_professional_school,education_doctoral,education_population,education_high_school_diploma,education_GED,education_some_college_less_than_year,...,age_median,aggregate_household_income,mean_household_income_lowest_quintile,mean_household_income_second_quintile,mean_household_income_third_quintile,mean_household_income_fourth_quintile,mean_household_income_highest_quintile,mean_household_income_top_5_percent,state_code,zcta
0,0.3893,1071,56,20,6,2,721,217,18,50,...,52.5,24224900.0,15735.0,28976.0,41584.0,60403.0,121871.0,186273.0,04,85610
1,0.4224,23777,5034,2577,645,498,21588,3984,416,1997,...,70.7,882515500.0,15092.0,33942.0,52059.0,78902.0,156740.0,271955.0,04,85614
2,0.4391,1289,226,164,46,0,897,222,16,10,...,51.1,30288800.0,12085.0,26596.0,40793.0,63481.0,139590.0,195939.0,04,85624
3,0.3565,25770,3482,1696,302,140,16121,3035,526,1320,...,35.2,784399700.0,24218.0,55100.0,82356.0,109225.0,187332.0,290584.0,04,85629
4,0.5500,1757,189,74,55,0,1395,349,39,116,...,58.3,53583600.0,6123.0,16639.0,37332.0,58660.0,167178.0,298053.0,04,85630


In [26]:
main_df.to_csv('../data/base_census.csv')

The next step is to calculate some additional variables. I want to keep this `main_df` as it is, so I will create a copy to work with.

In [27]:
census_df = main_df.copy()

The ACS did not offer average household income, so I will calculate it as `aggregate_household_income`/`household_count`

In [28]:
census_df['average_household_income'] = census_df['aggregate_household_income']/census_df['household_count']

### Education Variables

There are a few education columns I need to make from the columns I have.
- `education_college_level` -> any college up to Bachelors (associates and some college)
- `education_high_school_graduate` -> high school diploma or GED
- `education_less_than_highschool` -> total education population minus all of those who have at least graduated high school

Beyond that, I want to calculate the rates for each of these columns. For example, the rate of the population with a Bachelors will be `education_bachelors` / `education_population`. Same for the rest of them.

In [29]:
census_df['education_college'] = census_df['education_associates'] + census_df['education_some_college_less_than_year'] + census_df['education_some_college_no_degree']

In [30]:
census_df['education_high_school_graduate'] = census_df['education_high_school_diploma'] + census_df['education_GED']

In [31]:
census_df['education_less_than_high_school'] = census_df['education_population'] - (census_df['education_bachelor'] + census_df['education_college'] + census_df['education_doctoral'] + 
                                                                                    census_df['education_high_school_graduate'] + census_df['education_master'] + census_df['education_professional_school'])

Now I can calculate education rates for the 7 main variables

In [32]:
census_df['education_less_than_high_school_rate'] = census_df['education_less_than_high_school'] / census_df['education_population']
census_df['education_high_school_graduate_rate'] = census_df['education_high_school_graduate'] / census_df['education_population']
census_df['education_college_rate'] = census_df['education_college'] / census_df['education_population']
census_df['education_bachelor_rate'] = census_df['education_bachelor'] / census_df['education_population']
census_df['education_master_rate'] = census_df['education_master'] / census_df['education_population']
census_df['education_professional_school_rate'] = census_df['education_professional_school'] / census_df['education_population']
census_df['education_doctoral_rate'] = census_df['education_doctoral'] / census_df['education_population']

### Heating Fuel Variables

- `heating_fuel_coal_wood` -> coal/coke + wood
- `heating_fuel_all_gas` -> gas + bottled, tank, and LP gas

In [33]:
census_df['heating_fuel_coal_wood'] = census_df['heating_fuel_coal_coke'] + census_df['heating_fuel_wood']
census_df['heating_fuel_all_gas'] = census_df['heating_fuel_gas'] + census_df['heating_fuel_bottled_tank_lp_gas']

I also calculate the rate of each type of heating fuel. `heating_fuel_all_gas` / `housing_unit_count` for example. I do this for all main heating fuel variables.

In [34]:
census_df['heating_fuel_all_gas_rate'] = census_df['heating_fuel_all_gas'] / census_df['housing_unit_count']
census_df['heating_fuel_electricity_rate'] = census_df['heating_fuel_electricity'] / census_df['housing_unit_count']
census_df['heating_fuel_fuel_oil_kerosene_rate'] = census_df['heating_fuel_fuel_oil_kerosene'] / census_df['housing_unit_count']
census_df['heating_fuel_coal_wood_rate'] = census_df['heating_fuel_coal_wood'] / census_df['housing_unit_count']
census_df['heating_fuel_solar_rate'] = census_df['heating_fuel_solar'] / census_df['housing_unit_count']
census_df['heating_fuel_other_rate'] = census_df['heating_fuel_other'] / census_df['housing_unit_count']
census_df['heating_fuel_none_rate'] = census_df['heating_fuel_none'] / census_df['housing_unit_count']

### Age Variables

Currently the age columns are very specific and also broken into male and female, I want to group some of them together and then calculate the rate for each.

In [35]:
# age 18-24 rate
census_df['age_18_24_rate'] = (census_df['male_18_to_19'] + census_df['female_18_to_19'] + census_df['male_20'] + census_df['female_20'] +\
                                census_df['male_21'] + census_df['female_21'] + census_df['male_22_to_24'] + census_df['female_22_to_24']) / census_df['total_population']

In [36]:
# age 25-34 rate
census_df['age_25_35_rate'] = (census_df['male_25_to_29'] + census_df['female_25_to_29'] + census_df['male_30_to_34'] + census_df['female_30_to_34']) / census_df['total_population']

In [37]:
# age greater than 85 rate
census_df['age_more_than_85_rate'] = (census_df['male_85_over'] + census_df['female_85_over']) / census_df['total_population']

In [38]:
# age 75-84 rate
census_df['age_75_84_rate'] = (census_df['male_75_to_79'] + census_df['female_75_to_79'] + census_df['male_80_to_84'] + census_df['female_80_to_84']) / census_df['total_population']

In [39]:
# age 35-44 rate
census_df['age_35_44_rate'] = (census_df['male_35_to_39'] + census_df['female_35_to_39'] + census_df['male_40_to_44'] + census_df['female_40_to_44']) / census_df['total_population']

In [40]:
# age 45-54 rate
census_df['age_45_54_rate'] = (census_df['male_45_to_49'] + census_df['female_45_to_49'] + census_df['male_50_to_54'] + census_df['female_50_to_54']) / census_df['total_population']

In [41]:
# age 65-74 rate
census_df['age_65_74_rate'] = (census_df['male_65_to_66'] + census_df['female_65_to_66'] + census_df['male_67_to_69'] +\
                                census_df['female_67_to_69'] + census_df['male_70_to_74'] + census_df['female_70_to_74']) / census_df['total_population']

In [42]:
# age 55-64 rate
census_df['age_55_64_rate'] = (census_df['male_55_to_59'] + census_df['female_55_to_59'] + census_df['male_60_to_61'] +\
                                census_df['female_60_to_61'] + census_df['male_62_to_64'] + census_df['female_62_to_64']) / census_df['total_population']

In [43]:
# age 10-14 rate
census_df['age_10_14_rate'] = (census_df['male_10_to_14'] + census_df['female_10_to_14']) / census_df['total_population']

In [44]:
# age 15-17 rate
census_df['age_15_17_rate'] = (census_df['male_15_to_17'] + census_df['female_15_to_17']) / census_df['total_population']

In [45]:
# age 5-9 rate
census_df['age_5_9_rate'] = (census_df['male_5_to_9'] + census_df['female_5_to_9']) / census_df['total_population']

### Race Rates

In [46]:
census_df['race_white_rate'] = census_df['race_white'] / census_df['total_population']
census_df['race_black_africa_rate'] = census_df['race_black_africa'] / census_df['total_population']
census_df['race_india_alaska_rate'] = census_df['race_india_alaska'] / census_df['total_population']
census_df['race_asian_rate'] = census_df['race_asian'] / census_df['total_population']
census_df['race_islander_rate'] = census_df['race_islander'] / census_df['total_population']
census_df['race_other_rate'] = census_df['race_other'] / census_df['total_population']
census_df['race_two_more_rate'] = census_df['race_two_or_more'] / census_df['total_population']

### Transportation Rates

In [47]:
census_df['transportation_home_rate'] = census_df['transportation_worked_from_home'] / census_df['transportation_total']
census_df['transportation_car_alone_rate'] = census_df['transportation_drove_alone'] / census_df['transportation_total']
census_df['transportation_walk_rate'] = census_df['transportation_walked'] / census_df['transportation_total']
census_df['transportation_carpool_rate'] = census_df['transportation_carpool'] / census_df['transportation_total']
census_df['transportation_other_rate'] = census_df['transportation_other'] / census_df['transportation_total']
census_df['transportation_public_rate'] = census_df['transportation_public'] / census_df['transportation_total']

### Average years of education

Deep Solar calculated average years of education as the weighted sum of each education rate, where the rate is multiplied by typical number of years to obtain that degree. Please refer to `supplementary-info.pdf` for full equation.

In [48]:
census_df['average_years_of_education'] = (census_df['education_less_than_high_school_rate'] * 8) + (census_df['education_high_school_graduate_rate'] * 12) + \
    (census_df['education_college_rate'] * 14) + (census_df['education_bachelor_rate'] * 16) + (census_df['education_master_rate'] * 18) + \
        (census_df['education_professional_school_rate'] * 21) + (census_df['education_doctoral_rate'] * 21)

I may add a few more columns later on, but this will work for now. Lastly, I would like to get land and water area from the geographic json files. `STATES` contains all of the state names I need the data for, these are also the geojson file names in data/geography. I want to iterate over all of these geojson files, and collect the land and water area for all ZCTAs within each state. I will start first with Arkansas as an example.

In [50]:
with open('../data/geography/{}.json'.format(STATES[0])) as f:
    ak_codes = json.load(f)

In [51]:
ak_codes['features'][0]['properties']

{'STATEFP10': '04',
 'ZCTA5CE10': '86506',
 'GEOID10': '0486506',
 'CLASSFP10': 'B5',
 'MTFCC10': 'G6350',
 'FUNCSTAT10': 'S',
 'ALAND10': 440942496,
 'AWATER10': 212370,
 'INTPTLAT10': '+35.3580528',
 'INTPTLON10': '-109.2204013',
 'PARTFLG10': 'N'}

`ALAND10` and `AWATER10` are land and water area in meters squared respectively. I want to convert these to miles squared and also find total area as `land_area` + `water_area`.

Below I create a list of dictionaries called `zcta_areas` to store this data. Each dictionary will have the `STATEFP10`, `ZCTA5CE10`, `ALAND10`, and `AWATER10`

In [53]:
zcta_areas = []
# iterate over states
for state in STATES:

    # Open json file with geo data
    with open('../data/geography/{}.json'.format(state)) as f:
        geo_data = json.load(f)

    # Iterate over features
    for feat in geo_data['features']:

        # Extract properties
        props = feat['properties']

        # zcta dictonary
        zcta_dict = {}

        # State code
        zcta_dict['state_code'] = props['STATEFP10']

        # ZCTA
        zcta_dict['zcta'] = props['ZCTA5CE10']

        # land area
        zcta_dict['land_area'] = props['ALAND10']

        # Water area
        zcta_dict['water_area'] = props['AWATER10']

        # append dictionary to list
        zcta_areas.append(zcta_dict)

Create pandas dataframe from list of dictionaries

In [55]:
zcta_area_df = pd.DataFrame(zcta_areas)
zcta_area_df.head()

,state_code,zcta,land_area,water_area
0,04,86506,440942496,212370
1,04,86520,645145270,152731
2,04,86556,624152459,2441353
3,04,86545,846477596,117680
4,04,85638,838889403,216354


In [56]:
zcta_area_df.shape

(15768, 4)

In [57]:
census_df.shape

(15740, 158)

In [58]:
census_df['zcta'].duplicated().sum()

0

In [59]:
zcta_area_df['zcta'].duplicated().sum()

13

There are a few duplicates from the geographic data, it considers a few ZCTAs to belong to more than one state. I will join these two using the `census_df` ZCTA column as the key, as they are all unique values. First I want to convert meters squared to miles squared and calculate the `total_area`. The formula I am using for the conversion is m² * 0.00000038610

In [60]:
zcta_area_df['land_area'] = zcta_area_df['land_area'] * 0.00000038610
zcta_area_df['water_area'] = zcta_area_df['water_area'] * 0.00000038610
zcta_area_df['total_area'] = zcta_area_df['land_area'] + zcta_area_df['water_area']
zcta_area_df.head()

,state_code,zcta,land_area,water_area,total_area
0,04,86506,170.247898,0.081996,170.329894
1,04,86520,249.090589,0.058969,249.149558
2,04,86556,240.985264,0.942606,241.927871
3,04,86545,326.825000,0.045436,326.870436
4,04,85638,323.895198,0.083534,323.978733


In [61]:
merged_df = census_df.merge(zcta_area_df[['zcta', 'land_area', 'water_area', 'total_area']], how='left', on='zcta')

It added 16 extra rows. These must be the ZCTA duplicates.

In [62]:
merged_df[merged_df.duplicated(keep=False)][['zcta', 'state_code']]

,zcta,state_code
54,86044,04
55,86044,04
172,86514,04
173,86514,04
174,86515,04
175,86515,04
186,86504,04
187,86504,04
2022,97635,06
2023,97635,06


The state codes are the same for each, so I can drop the duplicates

In [63]:
merged_df = merged_df.drop_duplicates()

In [64]:
merged_df.shape

(15740, 161)

Lastly I need to drop the extra columns I don't intend to use for my analysis.

In [65]:
merged_df.columns

Index(['gini_index', 'total_population', 'education_bachelor',
       'education_master', 'education_professional_school',
       'education_doctoral', 'education_population',
       'education_high_school_diploma', 'education_GED',
       'education_some_college_less_than_year',
       ...
       'transportation_home_rate', 'transportation_car_alone_rate',
       'transportation_walk_rate', 'transportation_carpool_rate',
       'transportation_other_rate', 'transportation_public_rate',
       'average_years_of_education', 'land_area', 'water_area', 'total_area'],
      dtype='object', length=161)

Below I declare a list of columns I want to keep, which will be used to filter `merged_df`. First however, I want to save `merged_df` as a csv so I have it. I will save the filtered dataframe as a cleaner version of this as well

In [66]:
merged_df.to_csv('../data/full_census.csv')

In [67]:
cols_to_keep = ['zcta', 'state_code', 'average_household_income', 'mean_household_income_lowest_quintile', 'mean_household_income_second_quintile', 'mean_household_income_third_quintile', 'mean_household_income_fourth_quintile',
                'mean_household_income_highest_quintile', 'mean_household_income_top_5_percent', 'education_bachelor', 'education_college', 'education_doctoral', 'education_master', 'education_population', 'education_professional_school',
                'education_high_school_graduate', 'education_less_than_high_school', 'education_less_than_high_school_rate', 'education_high_school_graduate_rate', 'education_college_rate', 'education_bachelor_rate', 
                'education_master_rate', 'education_professional_school_rate', 'education_doctoral_rate', 'heating_fuel_coal_wood', 'heating_fuel_electricity', 'heating_fuel_fuel_oil_kerosene', 'heating_fuel_all_gas', 'heating_fuel_none',
                'heating_fuel_housing_unit_count', 'heating_fuel_other', 'heating_fuel_solar', 'race_asian', 'race_black_africa', 'race_india_alaska', 'race_islander', 'race_other', 'race_two_or_more', 'race_white', 'race_white_rate',
                'race_black_africa_rate', 'race_india_alaska_rate', 'race_asian_rate', 'race_islander_rate', 'race_other_rate', 'race_two_more_rate', 'heating_fuel_all_gas_rate', 'heating_fuel_electricity_rate',
                'heating_fuel_fuel_oil_kerosene_rate', 'heating_fuel_coal_wood_rate', 'heating_fuel_solar_rate', 'heating_fuel_other_rate', 'heating_fuel_none_rate', 'age_18_24_rate', 'age_25_35_rate', 'age_more_than_85_rate',
                'age_75_84_rate', 'age_35_44_rate', 'age_45_54_rate', 'age_65_74_rate', 'age_55_64_rate', 'age_10_14_rate', 'age_15_17_rate', 'age_5_9_rate', 'transportation_home_rate', 'transportation_car_alone_rate',
                'transportation_walk_rate', 'transportation_carpool_rate', 'transportation_other_rate', 'transportation_public_rate', 'age_median', 'employed', 'gini_index', 'land_area',
                'water_area', 'total_area', 'total_population', 'per_capita_income', 'median_household_income', 'household_count', 'housing_unit_count', 'housing_unit_occupied_count', 'housing_unit_median_value',
                'housing_unit_median_gross_rent', 'average_years_of_education']

In [68]:
len(cols_to_keep)

85

I should have 85 columns after filtering `merged_df`

In [69]:
filtered_df = merged_df[cols_to_keep]
filtered_df.shape

(15740, 85)

There were a few typos in my original column names. I can fix this later. For now I will adjust the column names above to match the existing ones.

There are just a couple more variables I need to calculate before saving this dataframe. I also want to adjust the mispelled column names, I will do that first.

In [70]:
NEW_COLUMN_NAMES = {'race_india_alaska': 'race_indian_alaska', 'race_india_alaska_rate': 'race_indian_alaska_rate', 'age_25_35_rate': 'age_25_34_rate', 'race_two_or_more': 'race_two_more'}

In [71]:
filtered_df = filtered_df.rename(columns=NEW_COLUMN_NAMES)

`population_density` is equal to `total_population` divided by `total_area`

In [72]:
filtered_df['population_density'] = filtered_df['total_population'] / filtered_df['total_area']

`employed` is full of NaN values. If I want to do anything with employment information I will need to try collecting the data again. For now, I will save this dataframe as a csv and move on to combining the data from all my sources.

In [73]:
filtered_df.to_csv('../data/census.csv')